In [1]:
import pandas as pd
import numpy as np
from scipy.spatial import cKDTree
from scipy.optimize import curve_fit
import warnings

warnings.filterwarnings("ignore")

In [2]:
main = pd.read_csv("data/total.csv")
water = pd.read_csv("data/water_velocity.csv")
wind = pd.read_csv("data/wind_velocity.csv")

In [3]:
main

,drifter,time,longitude,latitude
0,0,51.25,129.311005,34.950001
1,0,51.50,129.401993,35.020000
2,0,51.75,129.514008,35.061001
3,0,52.00,129.524002,35.032001
4,0,52.25,129.626007,35.083000
...,...,...,...,...
15840,30,638.00,147.348999,44.167999
15841,30,638.25,147.307007,44.138000
15842,30,638.50,147.250000,44.146999
15843,30,638.75,147.322998,44.201000


u : +는 동쪽, -는 서쪽<br>
v : +는 북쪽, -는 남쪽

In [6]:
water = water.fillna(0)
wind = wind.fillna(0)

In [7]:
main["water_u"] = 0
main["water_v"] = 0
main["wind_u"] = 0
main["wind_v"] = 0
main['d_long'] = main['longitude'].diff().fillna(0)
main['d_lati'] = main['latitude'].diff().fillna(0)

main = main.groupby('drifter').apply(lambda group: group.iloc[1:]).reset_index(drop=True)

In [8]:
main

,drifter,time,longitude,latitude,water_u,water_v,wind_u,wind_v,d_long,d_lati
0,0,51.50,129.401993,35.020000,0,0,0,0,0.090988,0.070000
1,0,51.75,129.514008,35.061001,0,0,0,0,0.112015,0.041000
2,0,52.00,129.524002,35.032001,0,0,0,0,0.009995,-0.028999
3,0,52.25,129.626007,35.083000,0,0,0,0,0.102005,0.050999
4,0,52.50,129.712997,35.058998,0,0,0,0,0.086990,-0.024002
...,...,...,...,...,...,...,...,...,...,...
15809,30,638.00,147.348999,44.167999,0,0,0,0,0.029999,-0.000999
15810,30,638.25,147.307007,44.138000,0,0,0,0,-0.041992,-0.029999
15811,30,638.50,147.250000,44.146999,0,0,0,0,-0.057007,0.008999
15812,30,638.75,147.322998,44.201000,0,0,0,0,0.072998,0.054001


In [11]:
water

,time,longitude,latitude,u,v
0,0.00,123.875,30.125,0.086603,-0.309205
1,0.00,124.125,30.125,0.067756,-0.295678
2,0.00,124.375,30.125,0.081384,-0.253029
3,0.00,124.625,30.125,0.104257,-0.347986
4,0.00,124.875,30.125,0.097815,-0.409650
...,...,...,...,...,...
25520795,729.75,145.625,53.625,0.014556,-0.133729
25520796,729.75,145.875,53.625,0.006857,-0.136531
25520797,729.75,146.125,53.625,-0.000076,-0.133501
25520798,729.75,146.375,53.625,-0.004371,-0.127829


In [9]:
# Create a grid of longitude and latitude from the water DataFrame
grid = water[['longitude', 'latitude']].drop_duplicates().values

# Create a KD-Tree for efficient nearest neighbor search
kdtree = cKDTree(grid)

# Function to find the 4 closest vertices for a given point
def find_closest_vertices(row):
    point = np.array([row['longitude'], row['latitude']])
    # Query the KD-Tree to find the 4 closest vertices
    _, indices = kdtree.query(point, k=4)
    return indices

# Apply the function to the main DataFrame to find the 4 closest vertices for each point
main['closest_vertices'] = main.apply(find_closest_vertices, axis=1)

In [10]:
main

,drifter,time,longitude,latitude,water_u,water_v,wind_u,wind_v,d_long,d_lati,closest_vertices
0,0,51.50,129.401993,35.020000,0,0,0,0,0.090988,0.070000,"[1862, 1770, 1863, 1771]"
1,0,51.75,129.514008,35.061001,0,0,0,0,0.112015,0.041000,"[1863, 1862, 1771, 1770]"
2,0,52.00,129.524002,35.032001,0,0,0,0,0.009995,-0.028999,"[1863, 1862, 1771, 1770]"
3,0,52.25,129.626007,35.083000,0,0,0,0,0.102005,0.050999,"[1863, 1771, 1864, 1862]"
4,0,52.50,129.712997,35.058998,0,0,0,0,0.086990,-0.024002,"[1863, 1864, 1771, 1772]"
...,...,...,...,...,...,...,...,...,...,...,...
15809,30,638.00,147.348999,44.167999,0,0,0,0,0.029999,-0.000999,"[5243, 5335, 5151, 5427]"
15810,30,638.25,147.307007,44.138000,0,0,0,0,-0.041992,-0.029999,"[5243, 5335, 5151, 5427]"
15811,30,638.50,147.250000,44.146999,0,0,0,0,-0.057007,0.008999,"[5243, 5335, 5151, 5427]"
15812,30,638.75,147.322998,44.201000,0,0,0,0,0.072998,0.054001,"[5243, 5335, 5151, 5427]"


In [6]:
u = []
v = []
for i in range(len(main)):
    x = water.loc[water["time"] == main.iloc[i][1]]
    w_u = []
    w_v = []
    w_d = []
    for j in main.iloc[i][-1]:
        d = ((main.iloc[i][2] - x.iloc[j][1])**2 + (main.iloc[i][3] - x.iloc[j][2])**2) ** 0.5
        w_u.append(x.iloc[j][-2])
        w_v.append(x.iloc[j][-1])
        w_d.append(d)

    # weithged average
    total_weight = sum(w_d)
    weighted_u = sum(w_u[i] * w_d[i] for i in range(len(w_d))) / total_weight
    weighted_v = sum(w_v[i] * w_d[i] for i in range(len(w_d))) / total_weight

    u.append(weighted_u)
    v.append(weighted_v)
    

In [7]:
main["water_u"] = u
main["water_v"] = v

In [10]:
# Create a grid of longitude and latitude from the water DataFrame
grid2 = wind[['longitude', 'latitude']].drop_duplicates().values

# Create a KD-Tree for efficient nearest neighbor search
kdtree = cKDTree(grid2)

# Function to find the 4 closest vertices for a given point
def find_closest_vertices(row):
    point = np.array([row['longitude'], row['latitude']])
    # Query the KD-Tree to find the 4 closest vertices
    _, indices = kdtree.query(point, k=4)
    return indices

# Apply the function to the main DataFrame to find the 4 closest vertices for each point
main['closest_vertices2'] = main.apply(find_closest_vertices, axis=1)

In [11]:
u = []
v = []
for i in range(len(main)):
    x = wind.loc[wind["time"] == main.iloc[i][1]]
    w_u = []
    w_v = []
    w_d = []
    for j in main.iloc[i][-1]:
        d = ((main.iloc[i][2] - x.iloc[j][1])**2 + (main.iloc[i][3] - x.iloc[j][2])**2) ** 0.5
        w_u.append(x.iloc[j][-2])
        w_v.append(x.iloc[j][-1])
        w_d.append(d)

    # weighted_average
    total_weight = sum(w_d)
    weighted_u = sum(w_u[i] * w_d[i] for i in range(len(w_d))) / total_weight
    weighted_v = sum(w_v[i] * w_d[i] for i in range(len(w_d))) / total_weight

    u.append(weighted_u)
    v.append(weighted_v)
    

In [12]:
main["wind_u"] = u
main["wind_v"] = v

In [13]:
main

,drifter,time,longitude,latitude,water_u,water_v,wind_u,wind_v,d_long,d_lati,closest_vertices,closest_vertices2
0,0,51.50,129.401993,35.020000,0.089673,0.041876,4.087482,-1.265760,0.090988,0.070000,"[1862, 1770, 1863, 1771]","[6830, 6738, 6831, 6739]"
1,0,51.75,129.514008,35.061001,0.095552,0.053962,3.988419,-1.597433,0.112015,0.041000,"[1863, 1862, 1771, 1770]","[6739, 6831, 6738, 6830]"
2,0,52.00,129.524002,35.032001,0.113870,0.135307,1.290056,0.333414,0.009995,-0.028999,"[1863, 1862, 1771, 1770]","[6831, 6739, 6830, 6738]"
3,0,52.25,129.626007,35.083000,0.259294,0.156671,5.640256,8.854036,0.102005,0.050999,"[1863, 1771, 1864, 1862]","[6739, 6831, 6740, 6738]"
4,0,52.50,129.712997,35.058998,0.368874,0.064946,13.505250,7.603525,0.086990,-0.024002,"[1863, 1864, 1771, 1772]","[6739, 6831, 6740, 6832]"
...,...,...,...,...,...,...,...,...,...,...,...,...
15809,30,638.00,147.348999,44.167999,0.076116,-0.035941,3.090491,1.099326,0.029999,-0.000999,"[5243, 5335, 5151, 5427]","[3495, 3587, 3403, 3679]"
15810,30,638.25,147.307007,44.138000,0.075121,-0.036927,1.551420,2.108921,-0.041992,-0.029999,"[5243, 5335, 5151, 5427]","[3495, 3587, 3403, 3679]"
15811,30,638.50,147.250000,44.146999,0.081807,-0.019462,1.289979,4.271356,-0.057007,0.008999,"[5243, 5335, 5151, 5427]","[3495, 3587, 3403, 3679]"
15812,30,638.75,147.322998,44.201000,0.081604,-0.005729,0.537444,4.900105,0.072998,0.054001,"[5243, 5335, 5151, 5427]","[3495, 3403, 3587, 3311]"


In [15]:
main.to_csv("data/total_add_velocity_weighted_average.csv", index = False)